In [5]:
#!/usr/bin/env python

def get_input(fname):
    r = []
    with open(fname) as f:
        for line in f.readlines():
            line = line.strip().split()
            r.append((line[1], line[7]))
    return r

In [6]:
test_input = get_input("test.txt")
print(test_input)

[('C', 'A'), ('C', 'F'), ('A', 'B'), ('A', 'D'), ('B', 'E'), ('D', 'E'), ('F', 'E')]


In [13]:
from collections import OrderedDict

def get_order(rules):
    before = OrderedDict()
    steps = sorted([r[0] for r in rules] + [r[1] for r in rules])
    for step in steps:
        before[step] = set()
    for b, a in rules:
        before[a].add(b)
    order = []
    while len(before) > 0:
        nxt = None
        for k in before:
            if len(before[k]) == 0:
                nxt = k
                break
        order.append(nxt)
        del before[nxt]
        for v in before.values():
            v.discard(nxt)
    return order

In [14]:
get_order(test_input)

['C', 'A', 'B', 'D', 'F', 'E']

In [16]:
actual_input = get_input("input.txt")
print(''.join(get_order(actual_input)))

BGJCNLQUYIFMOEZTADKSPVXRHW


In [34]:
def get_total_seconds(rules, number_of_workers=2, additional_seconds=0):
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    step_seconds = dict(zip(letters, range(1, 1 + len(letters))))
    total_seconds = 0
    before = OrderedDict()
    steps = sorted([r[0] for r in rules] + [r[1] for r in rules])
    for step in steps:
        before[step] = set()
    for b, a in rules:
        before[a].add(b)
    order = []
    counters = {}
    workers = [None for _ in range(number_of_workers)]
    while len(before) > 0:
        if not all(workers):
            nxts = []
            for k in before:
                if len(before[k]) == 0 and k not in workers:
                    nxts.append(k)
            if len(nxts) > 0:
                for nxt in nxts:
                    for i in range(number_of_workers):
                        if workers[i] is None:
                            workers[i] = nxt
                            counters[nxt] = step_seconds[nxt] + additional_seconds
                            break
#         print(counters, workers)
        for k in list(counters.keys()):
            v = counters[k]
            v -= 1
            if v == 0:
                del counters[k]
                for i in range(number_of_workers):
                    if workers[i] == k:
                        workers[i] = None
                del before[k]
                for v in before.values():
                    v.discard(k)
            else:
                counters[k] -= 1
        total_seconds += 1
    return total_seconds

In [35]:
print(get_total_seconds(test_input))

15


In [36]:
print(get_total_seconds(actual_input, number_of_workers=5, additional_seconds=60))

1017
